In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%pylab inline

In [ ]:
course_catalog_all = pd.read_csv('immuta/Combined Course Catalog/Combined Course Catalog.csv')

In [ ]:
series_url = course_catalog_all[['Series', 'Course Url']].drop_duplicates(inplace = False)
series_url.dropna()
invalid_url =  ["#VALUE!", "0", "null"]
series_url=series_url[series_url['Course Url'].isin(invalid_url)==False]
series_url = series_url.reset_index(drop = True)
len(series_url)


In [ ]:
url_desc = pd.read_csv('catalog_url_desc.csv')

In [ ]:
series_url_merged = pd.merge(series_url, url_desc, on = 'Course Url', how="left")
series_url_merged.head()

In [ ]:
len(series_url_merged)

In [ ]:
series_url_merged = series_url_merged.drop(['Unnamed: 0','Series_y','Course Title'], axis=1)

In [ ]:
series_url_merged.columns

In [ ]:

columns = [('_'.join(name.split())).lower() for name in series_url_merged.columns]
series_url_merged.columns = columns
series_url_merged.columns

In [ ]:
#string with all description
series_url_merged['all_desc'] = series_url_merged.coursedes + series_url_merged.target_audience + series_url_merged.prerequisites + series_url_merged.expected_duration + series_url_merged.lesson_objectives + series_url_merged.course_number

In [ ]:
#vectorize, row = document
vect= TfidfVectorizer(min_df=1)
series_url_merged['tfidf'] = vect.fit_transform(series_url_merged.all_desc)
series_url_merged.head()

In [ ]:
series_uniq = pd.unique(series_url_merged.series_x)

In [ ]:
def cosine_similarities_list(uniq_series):
    similarities = []
    for h in range(len(uniq_series)):
        indx = series_url_merged[['course_url']][series_url_merged.series_x==series_uniq[h]].index.tolist()
        docs =  series_url_merged[['all_desc']][series_url_merged.series_x==series_uniq[h]]
        doc_list= list(docs.values.flatten())
        vectors = vect.fit_transform(doc_list)
        for i in range(len(doc_list)):
            for j in range(len(doc_list)):
                if j>i:
                    cs = cosine_similarity(vectors[i],vectors[j])
                    similarities.append([0, indx[i], indx[j],round(cs.max(),5)])
                else:
                    pass
    return np.asarray(similarities)


In [ ]:
#similarities is a np array with pairwise cosine similarities of urls within the same series

similarities = cosine_similarities_list(series_uniq)


In [ ]:

similarities_df = pd.DataFrame(similarities)
similarities_df.columns = ['series','url1_id','url2_id','cs']

In [ ]:
similarities_df.cs.hist();

In [ ]:
len(similarities_df[similarities_df['cs']>0.9])

In [ ]:
similarities_df[similarities_df.cs == similarities_df.cs.max()]

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
series_url_merged.iloc[[144,943]]

In [ ]:
 similarities_90 = similarities_df[similarities_df.cs>0.9].sort_values('cs', ascending = False).reset_index(drop = True)

In [ ]:
series_url_merged.course_url.iloc[int(similarities_90.url1_id.iloc[1])]
#similarities_90.url1_id.iloc[1]

In [ ]:
url_pairs = [[]]
for i in range(len(similarities_90)):
    url_pairs.append([
        series_url_merged.course_url.iloc[int(similarities_90.url1_id.iloc[i])],
        series_url_merged.course_url.iloc[int(similarities_90.url2_id.iloc[i])]
                    ])
    
url_pairs #list with pairs of urls with top cosine similarities

### The url-pairs with the highest cosine similarities still refer to different classes. Some differences are in language (C# vs VBA) and level (Oracle Admin 1 vs II). Hence, all distinct URLs are unique classes